In [3]:
#pip install langchain langchain-community  langchain-openai langchain-experimental python-dotenv langchain_ollama reportlab

In [6]:
import os

In [5]:
openai_api = 'OPENAI_API_KEY'
os.environ["OPENAI_API_KEY"] = openai_api

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

class Agent:
    def __init__(self, medical_report=None, role=None, extra_info=None):
        self.medical_report = medical_report
        self.role = role
        self.extra_info = extra_info or {}  # Ensure extra_info is a dictionary
        # Initialize the prompt based on role and other info
        self.prompt_template = self.create_prompt_template()
        # Initialize the model
        self.model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

    def create_prompt_template(self):
        templates = {
            "Cardiologist": """
                Act like a cardiologist. You will receive a medical report of a patient.
                Task: Review the patient's cardiac workup, including ECG, blood tests, Holter monitor results, and echocardiogram.
                Focus: Determine if there are any subtle signs of cardiac issues that could explain the patient’s symptoms. Rule out any underlying heart conditions, such as arrhythmias or structural abnormalities, that might be missed on routine testing.
                Recommendation: Provide guidance on any further cardiac testing or monitoring needed to ensure there are no hidden heart-related concerns. Suggest potential management strategies if a cardiac issue is identified.
                Please only return the possible causes of the patient's symptoms and the recommended next steps.
                Medical Report: {medical_report}
            """,
            "Psychologist": """
                Act like a psychologist. You will receive a patient's report.
                Task: Review the patient's report and provide a psychological assessment.
                Focus: Identify any potential mental health issues, such as anxiety, depression, or trauma, that may be affecting the patient's well-being.
                Recommendation: Offer guidance on how to address these mental health concerns, including therapy, counseling, or other interventions.
                Please only return the possible mental health issues and the recommended next steps.
                Patient's Report: {medical_report}
            """,
            "Pulmonologist": """
                Act like a pulmonologist. You will receive a patient's report.
                Task: Review the patient's report and provide a pulmonary assessment.
                Focus: Identify any potential respiratory issues, such as asthma, COPD, or lung infections, that may be affecting the patient's breathing.
                Recommendation: Offer guidance on how to address these respiratory concerns, including pulmonary function tests, imaging studies, or other interventions.
                Please only return the possible respiratory issues and the recommended next steps.
                Patient's Report: {medical_report}
            """,
            "Neurologist": """
                Act like a neurologist. You will receive a patient's medical report.
                Task: Review the patient's neurological assessment, including CT scans, MRI results, and any other relevant tests.
                Focus: Identify any neurological disorders, such as strokes, epilepsy, or neurodegenerative diseases.
                Recommendation: Provide guidance on any further neurological testing, diagnosis, or potential treatments.
                Please return potential neurological diagnoses and next steps.
                Medical Report: {medical_report}
            """,
            "Endocrinologist": """
                Act like an endocrinologist. You will receive a medical report of a patient.
                Task: Review the patient's endocrine system assessment, including blood work related to thyroid, insulin, and cortisol levels.
                Focus: Identify potential endocrine disorders, such as diabetes, thyroid dysfunction, or adrenal disorders.
                Recommendation: Provide guidance on further testing, diagnosis, and treatment strategies.
                Please return possible endocrine-related diagnoses and next steps.
                Medical Report: {medical_report}
            """,
            "Immunologist": """
                Act like an immunologist. You will receive a medical report of a patient.
                Task: Review the patient's immune system evaluation, including tests for autoimmunity, allergies, or immunodeficiencies.
                Focus: Identify any immune system disorders, such as autoimmune diseases, immunodeficiencies, or allergic reactions.
                Recommendation: Provide guidance on further immune testing, diagnosis, or treatment plans.
                Please return possible immune-related disorders and recommended next steps.
                Medical Report: {medical_report}
            """,
            "MultidisciplinaryTeam": """
                Act like a multidisciplinary team of healthcare professionals.
                You will receive a medical report of a patient visited by a Cardiologist, Psychologist, Pulmonologist, Neurologist, Endocrinologist, and Immunologist.
                Task: Review the patient's medical reports from these specialists and analyze them to come up with a list of 3 possible health issues of the patient.
                Just return a list of bullet points of 3 possible health issues of the patient and for each issue provide the reason.
                
                Cardiologist Report: {cardiologist_report}
                Psychologist Report: {psychologist_report}
                Pulmonologist Report: {pulmonologist_report}
                Neurologist Report: {neurologist_report}
                Endocrinologist Report: {endocrinologist_report}
                Immunologist Report: {immunologist_report}
            """
        }

        if self.role not in templates:
            raise ValueError(f"Invalid role: {self.role}")

        return PromptTemplate.from_template(templates[self.role])

    def run(self):
        print(f"{self.role} is running...")
        try:
            if self.role == "MultidisciplinaryTeam":
                prompt = self.prompt_template.format(
                    cardiologist_report=self.extra_info.get("cardiologist_report", ""),
                    psychologist_report=self.extra_info.get("psychologist_report", ""),
                    pulmonologist_report=self.extra_info.get("pulmonologist_report", ""),
                    neurologist_report=self.extra_info.get("neurologist_report", ""),
                    endocrinologist_report=self.extra_info.get("endocrinologist_report", ""),
                    immunologist_report=self.extra_info.get("immunologist_report", "")
                )
            else:
                prompt = self.prompt_template.format(medical_report=self.medical_report)
            
            response = self.model.invoke(prompt)
            return response.content
        except Exception as e:
            print("Error occurred:", e)
            return None

# Define specialized agent classes for new agents
class Cardiologist(Agent):
    def __init__(self, medical_report):
        super().__init__(medical_report, "Cardiologist")

class Psychologist(Agent):
    def __init__(self, medical_report):
        super().__init__(medical_report, "Psychologist")

class Pulmonologist(Agent):
    def __init__(self, medical_report):
        super().__init__(medical_report, "Pulmonologist")

class Neurologist(Agent):
    def __init__(self, medical_report):
        super().__init__(medical_report, "Neurologist")

class Endocrinologist(Agent):
    def __init__(self, medical_report):
        super().__init__(medical_report, "Endocrinologist")

class Immunologist(Agent):
    def __init__(self, medical_report):
        super().__init__(medical_report, "Immunologist")

class MultidisciplinaryTeam(Agent):
    def __init__(self, cardiologist_report, psychologist_report, pulmonologist_report,
                 neurologist_report, endocrinologist_report, immunologist_report):
        extra_info = {
            "cardiologist_report": cardiologist_report,
            "psychologist_report": psychologist_report,
            "pulmonologist_report": pulmonologist_report,
            "neurologist_report": neurologist_report,
            "endocrinologist_report": endocrinologist_report,
            "immunologist_report": immunologist_report
        }
        super().__init__(role="MultidisciplinaryTeam", extra_info=extra_info)

In [9]:
import fitz  # PyMuPDF
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
import json, os

# Read the medical report from the PDF file
pdf_path = Path("sample_report_1.pdf")

if not pdf_path.exists():
    raise FileNotFoundError(f"The PDF report file '{pdf_path}' was not found.")

def read_pdf(file_path):
    # Open the PDF file
    doc = fitz.open(file_path)
    text = ""
    # Extract text from each page
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")  # Extract text from the page
    return text

# Read the medical report from the PDF
medical_report = read_pdf(pdf_path)

# Initialize agents for all specializations
agents = {
    "Cardiologist": Cardiologist(medical_report),
    "Psychologist": Psychologist(medical_report),
    "Pulmonologist": Pulmonologist(medical_report),
    "Neurologist": Neurologist(medical_report),
    "Endocrinologist": Endocrinologist(medical_report),
    "Immunologist": Immunologist(medical_report)
}

# Function to run each agent and get their response
def get_response(agent_name, agent):
    try:
        response = agent.run()
        return agent_name, response
    except Exception as e:
        return agent_name, f"Error: {str(e)}"

# Run the agents concurrently and collect responses
responses = {}
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_response, name, agent): name for name, agent in agents.items()}
    
    for future in as_completed(futures):
        agent_name, response = future.result()
        responses[agent_name] = response

# Check if all agents returned valid responses
if any("Error" in response for response in responses.values()):
    raise ValueError(f"One or more agents failed: {responses}")

# Create the MultidisciplinaryTeam agent with responses from all agents
team_agent = MultidisciplinaryTeam(
    cardiologist_report=responses["Cardiologist"],
    psychologist_report=responses["Psychologist"],
    pulmonologist_report=responses["Pulmonologist"],
    neurologist_report=responses["Neurologist"],
    endocrinologist_report=responses["Endocrinologist"],
    immunologist_report=responses["Immunologist"]
)

# Run the MultidisciplinaryTeam agent to generate the final diagnosis
try:
    final_diagnosis = team_agent.run()
    final_diagnosis_text = "### Final Diagnosis:\n\n" + final_diagnosis
except Exception as e:
    raise RuntimeError(f"MultidisciplinaryTeam agent failed: {str(e)}")

# Define the output file path
txt_output_path = Path("results/final_diagnosis_report.txt")

# Ensure the results directory exists
txt_output_path.parent.mkdir(parents=True, exist_ok=True)

# Write the final diagnosis to the text file
with txt_output_path.open("w") as txt_file:
    txt_file.write(final_diagnosis_text)

print(f"Final diagnosis has been saved to {txt_output_path}")

Cardiologist is running...Psychologist is running...

Pulmonologist is running...
Neurologist is running...
Endocrinologist is running...
Immunologist is running...
MultidisciplinaryTeam is running...
Final diagnosis has been saved to results/final_diagnosis_report.txt
